# PART I: Build, Train, and Test CNN on MNIST Dataset

Project Report – CNN on MNIST with TensorFlow.


In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [6]:
mnist.train.num_examples

55000

In [7]:
mnist.test.num_examples

10000

# Initialize Bias

In [8]:
def initialize_weights (filter_shape):
    
    init_random_dist=tf.truncated_normal(filter_shape, stddev=0.1)
    
    return(tf.Variable(init_random_dist))

In [9]:
def initialize_bias(bias_shape):
    
    initial_bias_value=tf.constant(0.1, shape=bias_shape)
    
    return(tf.Variable(initial_bias_value))

# Initialize Weights In Filter 

In [10]:
def create_convolution_layer_and_compute_dot_product(inputs, filter_shape):
    
    filter_initialized_with_weights=initialize_weights(filter_shape)
    
    conv_layer_outputs=tf.nn.conv2d(inputs, 
                                    filter_initialized_with_weights,
                                    strides=[1,1,1,1],
                                   padding="SAME")
    return(conv_layer_outputs)

In [11]:
def create_relu_layer_and_compute_dotproduct_plus_b(inputs, filter_shape):
    b=initialize_bias([filter_shape[3]])
    relu_layer_outputs=tf.nn.relu(inputs+b)
    return(relu_layer_outputs)

In [12]:
def create_maxpool2by2_and_reduce_spatial_size(inputs):
    pooling_layer_outputs=tf.nn.max_pool(inputs, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    return pooling_layer_outputs

In [13]:
def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs, output_size):
    input_size=int(inputs.get_shape()[1])
    w=initialize_weights([input_size, output_size])
    b=initialize_bias([output_size])
    fc_xW_plus_bias_outputs=tf.matmul(inputs, w)+b
    return(fc_xW_plus_bias_outputs)

# Create Place holders for inputs and Labels: x and y_true 

In [14]:
x=tf.compat.v1.placeholder(tf.float32, shape=[None, 784])

In [15]:
y_true=tf.compat.v1.placeholder(tf.float32, [None, 10])

In [16]:
x_image=tf.reshape(x, [-1,28,28,1])

In [17]:
conv_layer1_outputs \
    =create_convolution_layer_and_compute_dot_product(x_image, filter_shape=[5,5,1,32])
conv_relu_layer1_outputs \
    =create_relu_layer_and_compute_dotproduct_plus_b(conv_layer1_outputs, filter_shape=[5,5,1,32])

# Create 1st pooling layer and reduce spatial size 

In [18]:
# create the 1st pooling layyes
# then reduce the spatial size if the input data
# retrun outputs of the layer

pooling_layer_1_outputs=create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer1_outputs)

# Make it ready to be into 1 st full connected layer

In [19]:
# reshape and platten the output of the plling layer
# prepare to feed the output data into the frist fully connected layer

pooling_layer_1_outputs_flat=tf.reshape(pooling_layer_1_outputs, [-1,14*14*32])

In [20]:
conv_layer_2_outputs \
    =create_convolution_layer_and_compute_dot_product(pooling_layer_1_outputs, filter_shape=[5,5,32,64])
conv_relu_layer_2_outputs \
    =create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_2_outputs, filter_shape=[5,5,32,64])

In [21]:
pooling_layer_2_outputs=create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_2_outputs)

In [22]:
pooling_layer_2_outputs_flat=tf.reshape(pooling_layer_2_outputs, [-1,7*7*64])

In [23]:
fc_layer1_outputs=create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat, output_size=1024)

fc_relu_layer1_outputs=tf.nn.relu(fc_layer1_outputs)

In [24]:
hold_prob=tf.placeholder(tf.float32)

fc_dropout_outputs=tf.nn.dropout(fc_relu_layer1_outputs, keep_prob=hold_prob)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
y_pred=create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

In [26]:
softmax_cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
cross_entropy_mean=tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [27]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)

In [28]:
cnn_trainer=optimizer.minimize(cross_entropy_mean)

In [29]:
vars_initializer=tf.global_variables_initializer()
steps=5000

In [30]:
with tf.Session() as sess:    
    
    sess.run(vars_initializer)
    for i in range(steps):
        batch_x, batch_y=mnist.train.next_batch(50) 
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5}) 
        if i % 100 == 0:
            print("On step: {}".format(i))
            print("Accuracy: ")
            
            matches=tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc=tf.reduce_mean(tf.cast(matches, tf.float32))
            test_accuracy=sess.run(acc, feed_dict={x: mnist.test.images,
                                                   y_true: mnist.test.labels,
                                                   hold_prob: 1.0})      
            print(test_accuracy)
            print('\n')


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

On step: 0
Accuracy: 
0.1968


On step: 100
Accuracy: 
0.9449


On step: 200
Accuracy: 
0.9583


On step: 300
Accuracy: 
0.9686


On step: 400
Accuracy: 
0.9758


On step: 500
Accuracy: 
0.9785


On step: 600
Accuracy: 
0.9786


On step: 700
Accuracy: 
0.9808


On step: 800
Accuracy: 
0.9817


On step: 900
Accuracy: 
0.982


On step: 1000
Accuracy: 
0.9828


On step: 1100
Accuracy: 
0.9866


On step: 1200
Accuracy: 
0.9856


On step: 1300
Accuracy: 
0.9847


On step: 1400
Accuracy: 
0.9864


On step: 1500
Accuracy: 
0.9875


On step: 1600
Accuracy: 
0.9849


On step: 1700
Accuracy: 
0.9864


On step: 1800
Accuracy: 
0.9884


On step: 1900
Accuracy: 
0.9878


On step: 2000
Accuracy: 
0.9855


On step: 2100
Accuracy: 
0.9884


On step: 2200
Accuracy: 
0.9886


On step: 2300
Accuracy: 
0.9889


On step: 2400
Accuracy: 
0.9867


On step: 2500
Accuracy: 
0.9889


On step: 2600
Accuracy: 
0.9897


On step: 2700
Accuracy: 
0.987


On step: 2800
Accuracy: 
0.9857


On step: 2900
Accuracy: 
0.9